In [1]:
import pathpyG as pp
import torch 
import torch_geometric

In [2]:
g = pp.Graph.from_edge_list([('a', 'c'), ('b', 'c'), ('c', 'd'), ('c', 'e')])
print(g.mapping)
pp.plot(g, edge_color='gray')

a -> 0
c -> 1
b -> 2
d -> 3
e -> 4



In [3]:
pp.config['torch']['device'] = 'cuda'

In [4]:
n = 5
k = 2

paths = torch.nested.nested_tensor([[[0,1], [1,3]], [[2,1,4],[1,4,2]]], device=pp.config['torch']['device']).to_padded_tensor(padding=-1)
counts = torch.tensor([[1], [3]], device=pp.config['torch']['device'])
paths = paths.repeat(n, 1, 1)
counts = counts.repeat(n, 1).long()
print(paths)
print(counts)

tensor([[[ 0,  1, -1],
         [ 1,  3, -1]],

        [[ 2,  1,  4],
         [ 1,  4,  2]],

        [[ 0,  1, -1],
         [ 1,  3, -1]],

        [[ 2,  1,  4],
         [ 1,  4,  2]],

        [[ 0,  1, -1],
         [ 1,  3, -1]],

        [[ 2,  1,  4],
         [ 1,  4,  2]],

        [[ 0,  1, -1],
         [ 1,  3, -1]],

        [[ 2,  1,  4],
         [ 1,  4,  2]],

        [[ 0,  1, -1],
         [ 1,  3, -1]],

        [[ 2,  1,  4],
         [ 1,  4,  2]]], device='cuda:0')
tensor([[1],
        [3],
        [1],
        [3],
        [1],
        [3],
        [1],
        [3],
        [1],
        [3]], device='cuda:0')


/tmp/ipykernel_7220/2462390652.py:4: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  paths = torch.nested.nested_tensor([[[0,1], [1,3]], [[2,1,4],[1,4,2]]], device=pp.config['torch']['device']).to_padded_tensor(padding=-1)


In [5]:
x = paths.unfold(2, k, 1)
print(x)

w = counts.unfold(1,1,1).squeeze()
# w contains the counts of all higher-order edges (which possibly occur multiple times)
print(w)

tensor([[[[ 0,  1],
          [ 1, -1]],

         [[ 1,  3],
          [ 3, -1]]],


        [[[ 2,  1],
          [ 1,  4]],

         [[ 1,  4],
          [ 4,  2]]],


        [[[ 0,  1],
          [ 1, -1]],

         [[ 1,  3],
          [ 3, -1]]],


        [[[ 2,  1],
          [ 1,  4]],

         [[ 1,  4],
          [ 4,  2]]],


        [[[ 0,  1],
          [ 1, -1]],

         [[ 1,  3],
          [ 3, -1]]],


        [[[ 2,  1],
          [ 1,  4]],

         [[ 1,  4],
          [ 4,  2]]],


        [[[ 0,  1],
          [ 1, -1]],

         [[ 1,  3],
          [ 3, -1]]],


        [[[ 2,  1],
          [ 1,  4]],

         [[ 1,  4],
          [ 4,  2]]],


        [[[ 0,  1],
          [ 1, -1]],

         [[ 1,  3],
          [ 3, -1]]],


        [[[ 2,  1],
          [ 1,  4]],

         [[ 1,  4],
          [ 4,  2]]]], device='cuda:0')
tensor([1, 3, 1, 3, 1, 3, 1, 3, 1, 3], device='cuda:0')


In [6]:
ho_edge_index, inverse = x.unique(dim=0, return_inverse=True)
print(ho_edge_index)

# returns a list telling where elements in the input ended up in the unique output
print(inverse)

# weights of the two unique higher-order edges should be N and 3*N
# weights of k-th element in output = sum of all w at indices where inverse is k
weights = torch.zeros(ho_edge_index.size()[0], device=pp.config['torch']['device'], dtype=torch.long).index_add(0, inverse, w)
print(weights)

tensor([[[[ 0,  1],
          [ 1, -1]],

         [[ 1,  3],
          [ 3, -1]]],


        [[[ 2,  1],
          [ 1,  4]],

         [[ 1,  4],
          [ 4,  2]]]], device='cuda:0')
tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1], device='cuda:0')
tensor([ 5, 15], device='cuda:0')


In [7]:
# For a padded tensors, we must remove all HO-edges (and counts) where -1 occurs as a node

# get indices of all tensor elements that contain padded value
unpad = ho_edge_index < 0
print(unpad)
indices = unpad.nonzero()
print(indices)

tensor([[[[False, False],
          [False,  True]],

         [[False, False],
          [False,  True]]],


        [[[False, False],
          [False, False]],

         [[False, False],
          [False, False]]]], device='cuda:0')
tensor([[0, 0, 1, 1],
        [0, 1, 1, 1]], device='cuda:0')


In [8]:
indices[:,0]

tensor([0, 0], device='cuda:0')

In [9]:
indices[:,2]

tensor([1, 1], device='cuda:0')

we have negative entries at the two indices in dim=0 and dim=2 above, so the remaining three edges with non-negative nodes are the following 

In [64]:
ho_edge_index[0,:,0,:]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [58]:
ho_edge_index[1,:,0,:]

tensor([[2, 1],
        [1, 4]], device='cuda:0')

In [60]:
ho_edge_index[1,:,1,:]

tensor([[1, 4],
        [4, 2]], device='cuda:0')

In [47]:
# TODO: Remove all higher-order edges that contain a -1 (currently we instead remove all *tensors* in dim=0 that contain a -1, i.e. we also remove second-order edges that originate from padded tensors)
idx = torch.arange(ho_edge_index.size()[0], device=pp.config['torch']['device'])
if indices.size()[0]>0:    
    idx = idx[idx!=indices]
print(idx)

RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1

tensor([[0, 1],
        [1, 3]], device='cuda:0')

tensor([[2, 1],
        [1, 4]], device='cuda:0')

In [9]:
ho_edge_index = ho_edge_index[idx]
weights = weights[idx]
print(ho_edge_index)
print(weights)

tensor([[[[2, 1],
          [1, 4]],

         [[1, 4],
          [4, 2]]]], device='cuda:0')
tensor([15], device='cuda:0')


In [10]:
p = [
    torch.tensor([[0,1],                 
                 [1,3]], device=pp.config['torch']['device']),
    torch.tensor([[2,1,4],
                 [1,4,3]], device=pp.config['torch']['device']),

]
paths = pp.WalkDataNested(p, torch.tensor([1,5], device=pp.config['torch']['device']), mapping = g.mapping)
print(paths)

PathData with 2 walks and total weight 6


In [11]:
edge_index, weights = paths.edge_index_k_weighted(k=2)
print(edge_index)
print(weights)

tensor([[[[2, 1],
          [1, 4]],

         [[1, 4],
          [4, 3]]]], device='cuda:0')
tensor([5], device='cuda:0')
